In [1]:
import pandas as pd 
import os 
from glob import glob

#### 특정 경로에 있는 데이터 파일을 모두 하나로 결합하는 함수 
1. 특정 경로에 있는 파일의 목록을 로드 
2. 파일의 목록에서 특정 확장자를 필터링
3. 필터링된 목록을 기준으로 반복문을 실행
    - 해당하는 목록의 파일 하나씩을 데이터프레임으로 로드 
    - 비어있는 데이터프레임에 단순한 행 결합 
    - 만약에 로드중 인코딩 에러가 발생시 다른 engine을 이용
4. 결합된 데이터프레임을 되돌려준다. 

In [21]:
def read_df(_path, _encoding = 'utf-8'):
    # 해당하는 파일에서 확장자를 분리
    head, tail = os.path.splitext(_path)
    # head : ../csv/2021/202101_exepense_list
    # tail : .csv
    if tail == '.csv':
        df = pd.read_csv(_path, encoding= _encoding)
    elif tail == '.json':
        df = pd.read_json(_path, encoding= _encoding)
    elif tail == 'xml':
        df = pd.read_xml(_path, encoding=_encoding)
    elif tail in ['.xlsx', '.xls']:
        df = pd.read_excel(_path)
    else:
        df = pd.DataFrame()
    return df

In [24]:
read_df("../csv/2017/201701_expense_list.csv")

""


In [28]:
def data_load_add(_path, _end='csv'):
    # _path : 파일의 경로
    # _end : 파일의 확장자 

    # 파일의 경로를 기준으로 파일의 목록을 로드하여 변수에 저장 
    file_list = os.listdir(_path)
    _path += '/'
    # print(file_list)
    # 파일의 목록에서 _end의 확장자를 가진 파일의 목록을 제외하고는 모두 삭제(필터)
    # case1 : '.'을 기준으로 파일의 이름을 나눠준다. -> 마지막의 값을 확인 
    # print (file_list[0].split('.')[-1])
    # case2 : 문자에서 마지막에 csv라면
    # print(file_list[0].endswith(_end))
    # case3 : os 라이브러리를 이용해서 파일의 이름과 확장자를 나눠준다. 
    # print(os.path.splitext(file_list[0]))

    # 한줄로 for문을 이용하여 새로운 리스트 생성
    # [i for i in range(9) if i % 2 == 0] -> [0, 2, 4, 6, 8]
    file_list_filter = [
        file for file in file_list if file.split('.')[-1] == _end
    ]
    result = pd.DataFrame()
    # print(file_list_filter)
    # 반복문을 이용하여 데이터들을 로드하고 비어있는 데이터프레임에 단순한 행 결합
    for file in file_list_filter:
        try:
            df = read_df(_path+file)
        except:
            try:
                # read_df()함수 호출시 에러가 발생한다면
                df = read_df(_path+file, 'CP949')
            except:
                df = read_df(_path+file, 'EUC-KR')
        result = pd.concat([result, df])
    return result

In [ ]:
def data_load_add(_path, _end='csv'):
    file_list = os.listdir(_path)
    _path += '/'
    file_list_filter = [
        file for file in file_list if file.split('.')[-1] == _end
    ]
    result = pd.DataFrame()
    for file in file_list_filter:
        try:
            df = read_df(_path+file)
        except:
            try:
                # read_df()함수 호출시 에러가 발생한다면
                df = read_df(_path+file, 'CP949')
            except:
                df = read_df(_path+file, 'EUC-KR')
        result = pd.concat([result, df])
    return result

In [ ]:
# 엑셀 파일 로드시 에러 발생한다면 
# !pip install openpyxl

In [29]:
df_2021 = data_load_add('../csv/2021/')

In [34]:
# 데이터프레임을 파일로 저장 
# 데이터프레임이라는 타입을 csv, xlsx, json 타입으로 변경 
# to키워드 + 타입 
# csv 저장 -> to_csv, 엑셀 저장 -> to_excel
# index 매개변수 : 인덱스를 파일에 저장할것인가?
df_2021.to_csv('df_2021.csv', index=False)

In [ ]:
pd.read_csv('df_2021.csv')

#### 특정 경로에 있는 파일들을 데이터프레임으로 로드 하면서 각각 다른 변수에 저장하는 함수 
1. 특정 경로에 있는 파일의 목록을 로드 
2. 목록을 기준으로 반복문을 실행
    - 경로, 파일로 나눠준다. 
    - 파일을 이름과 확장자로 나눠준다. 
    - read_df 함수를 이용하여 데이터프레임을 로드 
    - 전역 변수의 이름으로 파일의 이름을 지정하여 데이터프레임을 대입 

In [36]:
for i in range(9):
    a = i

print(a)

8


In [ ]:
# 전역변수가 저장되어 있는 공간을 출력
globals()

In [38]:
globals()['b'] = 'test'

In [39]:
b

'test'

In [74]:
def data_load(_path):
    # 입력받은 경로를 기준으로 파일의 목록 로드 
    # glob
    file_list = glob(_path+"/*.*")
    # print(file_list)
    for file in file_list:
        # file에서 경로 부분과 파일 부분을 나눠준다. 
        folder, name = os.path.split(file)
        # print(folder, '\t' , name)
        # name에서 파일의 이름과 확장자로 나눠준다. 
        head, tail = os.path.splitext(name)
        print(head, '\t', tail)
        try : 
            df = read_df(file)
        except:
            try:
                df = read_df(file, 'CP949')
            except:
                df = read_df(file, 'EUC-KR')    
        # 전역변수에 head값을 사용한다. 
        globals()[head] = df.copy()

In [ ]:
data_load('../csv/2017')

In [80]:
globals()['201701_expense_list']

,nid,title,url,dept_nm_lvl_1,dept_nm_lvl_2,dept_nm_lvl_3,dept_nm_lvl_4,dept_nm_lvl_5,exec_yr,exec_month,expense_budget,expense_execution,category,dept_nm_full,exec_dt,exec_loc,exec_purpose,target_nm,payment_method,exec_amount
0,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-26 13:10,동해일식 (중구 무교동),기본소득과 장애인복지 논의간담회,장애인복지정책팀장 외 2명,카드,76000
1,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-25 22:41,김앤장 (중구 무교로),장애인단체 활동지원 논의간담회,장애인복지정책과장 외 3명,카드,102000
2,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-24 12:35,왕왕생고기 (중구 을지로),장애인 기본돌봄 복지시책 논의간담회,장애인복지정책팀장외7명,카드,80000
3,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-24 12:23,서울불고기 (중구 남대문로),서울시일자리통합지원센터 운영개선 논의간담회,장애인복지정책과장외5명,카드,112000
4,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-23 15:10,서울시청신매점,부서운영에 필요한 음료수 구매,장애인복지정책과직원,카드,16000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-11 13:01,차이79 종로구 새문안로5길,금융기관 보조금 지원계획수립관련 간담회 개최 비용지출,투자유치과장외5명,카드,27000
5513,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-06 12:58,강가 서울 중구 세종대로136,핀테크 산업지원시설 입주 관련 간담회 개최비용지출,금융협력관외6명,카드,70000
5514,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-04 12:47,이복만두 중구 무교동 27,F사 투자유치 협의를 위한 간담회 개최 비용지출,투자유치과장외4명,카드,57000
5515,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-03 12:42,불고기브라더스 중구 태평로 1가,핀테크사업 전략수립을 위한 간담회 개최비용지출,경제진흥본부장외10명,카드,152400
